In [85]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats

# Import Sheet as csv 

In [14]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

# Format sheet 

In [15]:
ps = ps[["Date", "User", "File", "Post patch?", "Notes", "Conc.", ">400", "Ratio", "SM_QC_PF", "Bad dates"]]
ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["Conc."] = pd.to_numeric(ps["Conc."].str.replace(' ',''), errors ='force')
ps[">400"] = pd.to_numeric(ps[">400"].str.replace(' ',''), errors ='force')
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
ps.tail()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
5670,2017-06-20,P8,Slc32a1-IRES-Cre;Ai14-326814.04.02.02,Nucleated,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",378.6,356.0,0.94,fail,NaN
5671,2017-06-20,P8,Slc32a1-IRES-Cre;Ai14-326814.04.02.03,No-Seal,"Cell Dimmed,Cell Shrunk Small Bubbles Seal got...",708.5,271.0,0.38,fail,NaN
5672,2017-06-20,P8,Slc32a1-IRES-Cre;Ai14-326814.04.02.04,Partial-Nucleus,"Cell Dimmed,Cell Shrunk Small Bubbles Max seal...",2316.2,1330.0,0.57,pass,NaN
5675,2017-06-20,P2,Slc32a1-IRES-Cre;Ai14-326814.02.02.01,Nucleated,Fluorescence in Pipette Small Bubbles,5776.2,3666.0,0.63,pass,NaN
5676,2017-06-20,P2,Slc32a1-IRES-Cre;Ai14-326814.02.02.02,Nucleated,Fluorescence in Pipette Small Bubbles,2814.0,1469.0,0.52,pass,NaN


In [16]:
ps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3171 entries, 3 to 5676
Data columns (total 10 columns):
Date           3171 non-null datetime64[ns]
User           3166 non-null category
File           3170 non-null object
Post patch?    3171 non-null category
Notes          1724 non-null object
Conc.          3084 non-null float64
>400           3159 non-null float64
Ratio          3084 non-null object
SM_QC_PF       3171 non-null category
Bad dates      870 non-null object
dtypes: category(3), datetime64[ns](1), float64(2), object(4)
memory usage: 207.7+ KB


# Filter out controls and contaminated samples

In [17]:
#may be redundant as controls are getting filtered out based on dropping na values from "Post patch?" column
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["Bad dates"] != "x"]
#ps = ps[mask]
#ps.info()
ps.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
3,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.00,0.81,pass,NaN
16,2016-04-06,P5,Ndnf - brl_160406_03,Outside-Out,wiped with kim wipe without water before loadi...,100.61,37.65,NaN,fail,NaN
17,2016-04-06,P5,Ndnf - brl_160406_04,Nucleated,NaN,209.76,167.00,0.69,pass,NaN
18,2016-04-06,P5,Ndnf - brl_160406_05,Partial-Nucleus,cell swelled,125.58,92.00,0.61,pass,NaN
20,2016-04-06,P5,Ndnf - brl_160406_07,partial Nucleated,NaN,135.18,128.00,0.87,pass,NaN


# Percentage of nucleated patches passing SM_QC 

In [18]:
mask = ps["Post patch?"] == "Nucleated"
mask
nuc = ps[mask]
nuc
nuc_count = nuc["Post patch?"].count()
nuc_count

542

In [19]:
ps["Post patch?"].value_counts()

Outside-Out                       867
No-Seal                           654
Nucleated                         542
Partial-Nucleus                   311
Entire-Cell                        29
no-seal                             5
Entire-Cell?                        3
partial Nucleated                   2
no -seal                            1
n                                   1
missing                             1
leaky Outside-Out                   1
outside-out                         1
Outside-Out?                        1
Nucleated patch                     1
Nucleated (maybe Entire-Cell?)      1
nucelated                           1
nucleus                             1
dtype: int64

In [20]:
pas = ps["SM_QC_PF"] == "pass"
pas.head()

3      True
16    False
17     True
18     True
20     True
Name: SM_QC_PF, dtype: bool

In [21]:
np = ps[mask & pas]
np.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
17,2016-04-06,P5,Ndnf - brl_160406_04,Nucleated,NaN,209.76,167.0,0.69,pass,NaN
69,2016-04-12,P5,Rorb - brl_160412_01,Nucleated,"couldnt see flourescence - cell too deep, but ...",195.16,183.0,0.87,pass,no slide in LIMS
109,2016-04-18,P5,Chat - brl_160418_02,Nucleated,"coudlnt get gigaseal, put still saw flur in pi...",135.75,116.0,0.85,pass,NaN
112,2016-04-18,P5,Chat - brl_160418_05,Nucleated,lost seal at 3 mins but still able to pull ou...,158.45,113.0,0.62,pass,NaN
125,2016-04-19,P1,Chat - kjh_160419_03,Nucleated,saw fluorescence in pipette,423.02,203.0,0.48,pass,NaN


In [22]:
nuc_pass = np["Post patch?"].count()
nuc_pass
#np["Post patch?"].count()

434

In [23]:
float(nuc_pass)/nuc_count

0.8007380073800738

# Bubbles Analysis 

In [67]:
bubbles = ps.dropna(subset = ["Notes"])
bubbles.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
3,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.00,0.81,pass,NaN
16,2016-04-06,P5,Ndnf - brl_160406_03,Outside-Out,wiped with kim wipe without water before loadi...,100.61,37.65,NaN,fail,NaN
18,2016-04-06,P5,Ndnf - brl_160406_05,Partial-Nucleus,cell swelled,125.58,92.00,0.61,pass,NaN
29,2016-04-07,P1,Ndnf - kjh_160407_03,Nucleated patch,"cell swelled, no fluorescence visible in pipette",114.09,96.00,0.73,pass,NaN
30,2016-04-07,P1,Ndnf - kjh_160407_04,Partial-Nucleus,"cell swelled a lot, no fluorescence visible in...",129.23,111.00,0.79,pass,NaN


# No Bubbles Analysis

In [25]:
mask2 = bubbles["Notes"].str.contains("No Bubbles")
no_bubbles = bubbles[mask2]
no_bubbles.head(3)
nb = no_bubbles["Notes"].count()
nb
#no_bubbles[">400"].sort_values()

74

# Mean Conc. (>400bp) No Bubbles (pg/ul) - pass and fail 

In [92]:
print "No Bubbles - pass and fail: concentration in pg/ul of fragments >400 b.p.\n"
count = no_bubbles[">400"].count()
print "Average =", no_bubbles[">400"].mean()
std_dev = no_bubbles[">400"].std()
print "Standard Deviation =", std_dev
print "Std Error of Mean = ", std_dev / np.sqrt(count)
print "Max =", no_bubbles[">400"].max()
print "Median =", no_bubbles[">400"].median()
print "Min =", no_bubbles[">400"].min()
#no_bubbles[">400"].value_counts()

No Bubbles - pass and fail: concentration in pg/ul of fragments >400 b.p.

Average = 1062.26027397
Standard Deviation = 722.270167739
Std Error of Mean =  84.5353290177
Max = 3341.0
Median = 914.0
Min = 82.0


# Mean Conc. (>400bp) No Bubbles (pg/ul) - passing only

In [93]:
no_bubbles_pass = no_bubbles[no_bubbles["SM_QC_PF"] == "pass"]
count = no_bubbles_pass[">400"].count()
print "No Bubbles - pass only: concentration in pg/ul of fragments >400 b.p.\n"
print "Average =", no_bubbles_pass[">400"].mean()
std_dev = no_bubbles_pass[">400"].std()
print "Standard Deviation =", std_dev
print "Std Error of Mean =", std_dev / np.sqrt(count)
print "Max =", no_bubbles_pass[">400"].max()
print "Median =", no_bubbles_pass[">400"].median()
print "Min =", no_bubbles_pass[">400"].min()
#no_bubbles[">400"].value_counts()

No Bubbles - pass only: concentration in pg/ul of fragments >400 b.p.

Average = 1166.35384615
Standard Deviation = 696.842262253
Std Error of Mean = 86.4326450491
Max = 3341.0
Median = 1026.0
Min = 216.0


In [28]:
no_bubbles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 5216 to 5662
Data columns (total 10 columns):
Date           74 non-null datetime64[ns]
User           74 non-null category
File           74 non-null object
Post patch?    74 non-null category
Notes          74 non-null object
Conc.          73 non-null float64
>400           73 non-null float64
Ratio          73 non-null object
SM_QC_PF       74 non-null category
Bad dates      0 non-null object
dtypes: category(3), datetime64[ns](1), float64(2), object(4)
memory usage: 5.1+ KB


In [29]:
bubb_pass = bubbles["SM_QC_PF"] == 'pass'
nb_pass = bubbles[mask2 & bubb_pass]
nb_pass.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
5216,2017-05-16,P2,Gad2-IRES-Cre;Ai14-318382.03.02.02,Nucleated,No Bubbles,904.0,644.0,0.71,pass,NaN
5217,2017-05-16,P2,Gad2-IRES-Cre;Ai14-318382.03.02.03,Nucleated,No Bubbles,760.2,570.0,0.75,pass,NaN
5240,2017-05-17,P2,Gad2-IRES-Cre;Ai14-318381.03.02.01,Partial-Nucleus,No Bubbles got nucleus and gigaohm seal but th...,2569.0,750.0,0.29,pass,NaN
5242,2017-05-17,P2,Gad2-IRES-Cre;Ai14-318381.03.02.03,Outside-Out,No Bubbles lost seal on cell during extraction...,758.6,608.0,0.8,pass,NaN
5244,2017-05-17,P2,Gad2-IRES-Cre;Ai14-318381.04.02.02,Nucleated,No Bubbles,644.3,423.0,0.66,pass,NaN


# No bubbles pass percentage 

In [30]:
'''No bubbles pass percentage'''
nb_p = nb_pass["Notes"].count()
nb_p
nb_pp = float(nb_p) / nb
nb_pp

0.8918918918918919

# Small Bubbles Analysis

In [71]:
mask3 = bubbles["Notes"].str.contains("Small Bubbles")
#mask3
sm_bubbles = bubbles[mask3]
sm_bubbles
sb = sm_bubbles["Notes"].count()
sb
#sm_bubbles["Conc."].sort_values()
#sm_bubbles.head()

184

# Mean Conc. (>400bp) Small Bubbles (pg/ul) - pass and fail 

In [94]:
sm_bubbles = sm_bubbles[sm_bubbles[">400"] < (10000.0)]
count = sm_bubbles[">400"].count()
print "Small Bubbles - pass and fail: concentration in pg/ul of fragments >400 b.p.\n"
print "Average =", sm_bubbles[">400"].mean()
std_dev = sm_bubbles[">400"].std()
print "Standard Deviation =", std_dev
print "Standard Error of Mean =", std_dev / np.sqrt(count)
print "Max =", sm_bubbles[">400"].max()
print "Median =", sm_bubbles[">400"].median()
print "Min =", sm_bubbles[">400"].min()
#sm_bubbles_pass[">400"].value_counts()
#sm_bubbles_pass[">400"].sort_values()


Small Bubbles - pass and fail: concentration in pg/ul of fragments >400 b.p.

Average = 933.166666667
Standard Deviation = 715.337905074
Standard Error of Mean = 53.3181394209
Max = 4990.0
Median = 756.5
Min = 73.0


# Mean Conc. (>400bp) Small Bubbles (pg/ul) - passing only

In [95]:
sm_bubbles_pass = sm_bubbles[sm_bubbles["SM_QC_PF"] == "pass"]
sm_bubbles_pass = sm_bubbles_pass[sm_bubbles_pass[">400"] < (10000.0)]
count = sm_bubbles_pass[">400"].count()
print "Small Bubbles - pass only: concentration in pg/ul of fragments >400 b.p.\n"
print "Average =", sm_bubbles_pass[">400"].mean()
std_dev = sm_bubbles_pass[">400"].std()
print "Standard Deviation =", std_dev
print "Standard Error of Mean =", std_dev / np.sqrt(count)
print "Max =", sm_bubbles_pass[">400"].max()
print "Median =", sm_bubbles_pass[">400"].median()
print "Min =", sm_bubbles_pass[">400"].min()
#sm_bubbles_pass[">400"].value_counts()
#sm_bubbles_pass[">400"].sort_values()

Small Bubbles - pass only: concentration in pg/ul of fragments >400 b.p.

Average = 1125.09285714
Standard Deviation = 699.145674927
Standard Error of Mean = 59.0885941839
Max = 4990.0
Median = 930.0
Min = 229.0


In [96]:
sm_bubbles.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 5209 to 5676
Data columns (total 10 columns):
Date           180 non-null datetime64[ns]
User           180 non-null category
File           180 non-null object
Post patch?    180 non-null category
Notes          180 non-null object
Conc.          180 non-null float64
>400           180 non-null float64
Ratio          180 non-null object
SM_QC_PF       180 non-null category
Bad dates      0 non-null object
dtypes: category(3), datetime64[ns](1), float64(2), object(4)
memory usage: 12.0+ KB


In [41]:
sb_pass = bubbles[mask3 & bubb_pass]
sb_pass.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
5211,2017-05-16,P1,Gad2-IRES-Cre;Ai14-318382.04.01.03,Nucleated,Small Bubbles,713.7,504.0,0.71,pass,NaN
5215,2017-05-16,P2,Gad2-IRES-Cre;Ai14-318382.03.02.01,Outside-Out,Small Bubbles,1338.9,758.0,0.57,pass,NaN
5228,2017-05-17,P1,Gad2-IRES-Cre;Ai14-318381.03.01.01,Nucleated,"Small Bubbles,Medium Bubbles",866.1,580.0,0.67,pass,NaN
5229,2017-05-17,P1,Gad2-IRES-Cre;Ai14-318381.03.01.02,Partial-Nucleus,Small Bubbles,382.8,229.0,0.6,pass,NaN
5230,2017-05-17,P1,Gad2-IRES-Cre;Ai14-318381.03.01.03,Nucleated,Small Bubbles,1712.6,1197.0,0.7,pass,NaN


In [42]:
sb_pass.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 5211 to 5676
Data columns (total 10 columns):
Date           144 non-null datetime64[ns]
User           144 non-null category
File           144 non-null object
Post patch?    144 non-null category
Notes          144 non-null object
Conc.          141 non-null float64
>400           141 non-null float64
Ratio          141 non-null object
SM_QC_PF       144 non-null category
Bad dates      0 non-null object
dtypes: category(3), datetime64[ns](1), float64(2), object(4)
memory usage: 9.7+ KB


# Small bubbles pass percentage

In [43]:
sb_p = sb_pass["Notes"].count()
sb_pp = float(sb_p) / sb
sb_pp

0.782608695652174

# Medium Bubbles Analysis

In [44]:
mask4 = bubbles["Notes"].str.contains("Medium Bubbles")
med_bubbles = bubbles[mask4]
med_bubbles
mb = med_bubbles["Notes"].count()
mb
#med_bubbles["Conc."].sort_values()
#med_bubbles.sort_values("Conc.")

50

# Mean Conc. (>400bp) Med Bubbles (pg/ul) - pass and fail 

In [97]:
print "Medium Bubbles: concentration in pg/ul of fragments >400 b.p.\n"
count = med_bubbles[">400"].count()
print "Average =", med_bubbles[">400"].mean()
std_dev = med_bubbles[">400"].std()
print "Standard Deviation =", std_dev
print "Standard Error of Mean =", std_dev / np.sqrt(count)
print "Max =", med_bubbles[">400"].max()
print "Median =", med_bubbles[">400"].median()
print "Min =", med_bubbles[">400"].min()
#med_bubbles[">400"].value_counts()

Medium Bubbles: concentration in pg/ul of fragments >400 b.p.

Average = 932.58
Standard Deviation = 650.155086365
Standard Error of Mean = 91.9458140784
Max = 2651.0
Median = 756.5
Min = 73.0


# Mean Conc. (>400bp) Med Bubbles (pg/ul) - passing only

In [98]:
med_bubbles_pass = med_bubbles[med_bubbles["SM_QC_PF"] == "pass"]
count = med_bubbles_pass[">400"].count()
print "Medium Bubbles: concentration in pg/ul of fragments >400 b.p.\n"
print "Average =", med_bubbles_pass[">400"].mean()
std_dev = med_bubbles_pass[">400"].std()
print "Standard Deviation =", std_dev / np.sqrt(count)
print "Max =", med_bubbles_pass[">400"].max()
print "Median =", med_bubbles_pass[">400"].median()
print "Min =", med_bubbles_pass[">400"].min()
#med_bubbles[">400"].value_counts()

Medium Bubbles: concentration in pg/ul of fragments >400 b.p.

Average = 1098.04878049
Standard Deviation = 93.8377033748
Max = 2651.0
Median = 843.0
Min = 339.0


In [47]:
mb_pass = bubbles[mask4 & bubb_pass]
mb_pass.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
5228,2017-05-17,P1,Gad2-IRES-Cre;Ai14-318381.03.01.01,Nucleated,"Small Bubbles,Medium Bubbles",866.1,580.0,0.67,pass,NaN
5236,2017-05-17,P1,Gad2-IRES-Cre;Ai14-318381.05.01.05,Nucleated,"Small Bubbles,Medium Bubbles",919.8,697.0,0.76,pass,NaN
5262,2017-05-18,P1,Pvalb-IRES-Cre;Ai14-316465.04.01.03,Outside-Out,"Cell Dimmed Small Bubbles,Medium Bubbles",813.3,517.0,0.64,pass,NaN
5300,2017-05-19,P1,Gad2-IRES-Cre;Ai14-318383.04.02.03,No-Seal,"Fluorescence in Pipette,Cell Dimmed,Cell Shrun...",641.8,457.0,0.71,pass,NaN
5302,2017-05-19,P1,Gad2-IRES-Cre;Ai14-318383.04.02.05,Nucleated,Cell Dimmed Medium Bubbles,748.5,597.0,0.8,pass,NaN


# Medium bubbles pass percentage

In [48]:
mb_p = mb_pass["Notes"].count()
mb_pp = float(mb_p)/ mb
mb_pp

0.82

# Large Bubbles Analysis

In [49]:
mask5 = bubbles["Notes"].str.contains("Large Bubbles")
lg_bubbles = bubbles[mask5]
lg_bubbles
lb = lg_bubbles["Notes"].count()
lb
#lg_bubbles["Conc."].sort_values()
#lg_bubbles.info()

20

# Mean Conc. (>400bp) Large Bubbles (pg/ul) - pass and fail 

In [102]:
print "Large Bubbles: concentration in pg/ul of fragments >400 b.p.\n"
count = lg_bubbles[">400"].count()
print "Average =", lg_bubbles[">400"].mean()
std_dev = lg_bubbles[">400"].std()
print "Standard Deviation =", std_dev
print "Standard Error of Mean =", std_dev / np.sqrt(count)
print "Max =", lg_bubbles[">400"].max()
print "Median =", lg_bubbles[">400"].median()
print "Min =", lg_bubbles[">400"].min()
#lg_bubbles[">400"].value_counts()

 Large Bubbles: concentration in pg/ul of fragments >400 b.p.

Average = 834.0
Standard Deviation = 732.746280176
Standard Error of Mean = 168.103525607
Max = 3041.0
Median = 827.0
Min = 73.0


# Mean Conc. (>400bp) Large Bubbles (pg/ul) - passing only

In [105]:
lg_bubbles_pass = lg_bubbles[lg_bubbles["SM_QC_PF"] == "pass"]
count = lg_bubbles_pass[">400"].count()
print "Large Bubbles: concentration in pg/ul of fragments >400 b.p.\n"
print "Average =", lg_bubbles_pass[">400"].mean()
std_dev = lg_bubbles_pass[">400"].std()
print "Standard Deviation =", std_dev
print "Standard Error of Mean =", std_dev / np.sqrt(count)
print "Max =", lg_bubbles_pass[">400"].max()
print "Median =", lg_bubbles_pass[">400"].median()
print "Min =", lg_bubbles_pass[">400"].min()
#lg_bubbles[">400"].value_counts()

Large Bubbles: concentration in pg/ul of fragments >400 b.p.

Average = 1140.23076923
Standard Deviation = 692.245880431
Standard Error of Mean = 191.994462856
Max = 3041.0
Median = 934.0
Min = 450.0


In [78]:
lb_pass = bubbles[mask5 & bubb_pass]
lb_pass.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
5235,2017-05-17,P1,Gad2-IRES-Cre;Ai14-318381.05.01.04,Nucleated,Large Bubbles,1086.7,827.0,0.76,pass,NaN
5243,2017-05-17,P2,Gad2-IRES-Cre;Ai14-318381.04.02.01,Nucleated,"Large Bubbles,Solution in Pipette Shank",988.1,494.0,0.5,pass,NaN
5277,2017-05-18,P2,Pvalb-IRES-Cre;Ai14-316465.06.01.01,Partial-Nucleus,Fluorescence in Pipette Large Bubbles,1267.7,888.0,0.7,pass,NaN
5347,2017-05-23,P2,Slc32a1-IRES-Cre;Ai14-321268.04.02.02,No-Seal,Fluorescence in Pipette Large Bubbles may have...,3641.2,3041.0,0.84,pass,NaN
5355,2017-05-23,P2,Slc32a1-IRES-Cre;Ai14-321268.05.02.05,Nucleated,Fluorescence in Pipette Large Bubbles,1984.1,1100.0,0.55,pass,NaN


# Large Bubbles pass percentage 

In [106]:
lb_p = lb_pass["Notes"].count()
lb_p
lb_pp = float(lb_p)/lb
lb_pp

0.7

# Solution up Pipette Shank Analysis 

In [80]:
mask6 = bubbles["Notes"].str.contains("Solution in Pipette Shank")
sol_pip = bubbles[mask6]
sol_pip
sp = sol_pip["Notes"].count()
sp

13

# Mean Conc. (>400bp) Solution up Pipette (pg/ul) - pass and fail 

In [107]:
print "Solution up Pipette Shank: concentration in pg/ul of fragments >400 b.p.\n"
count = sol_pip[">400"].count()
print "Average =", sol_pip[">400"].mean()
std_dev = sol_pip[">400"].std()
print "Standard Deviation =", std_dev
print "Standard Error of Mean =", std_dev / np.sqrt(count)
print "Max =", sol_pip[">400"].max()
print "Median =", sol_pip[">400"].median()
print "Min =", sol_pip[">400"].min()
#lg_bubbles[">400"].value_counts()

Solution up Pipette Shank: concentration in pg/ul of fragments >400 b.p.

Average = 377.833333333
Standard Deviation = 355.12656728
Standard Error of Mean = 102.516209608
Max = 1368.0
Median = 276.5
Min = 73.0


# Mean Conc. (>400bp) Solution up Pipette (pg/ul) - passing only

In [108]:
sol_pip_pass = sol_pip[sol_pip["SM_QC_PF"] == "pass"]
count = sol_pip_pass[">400"].count()
print "Solution up Pipette Shank: concentration in pg/ul of fragments >400 b.p.\n"
print "Average =", sol_pip_pass[">400"].mean()
std_dev = sol_pip_pass[">400"].std()
print "Standard Deviation =", std_dev
print "Standard Error of Mean =", std_dev / np.sqrt(count)
print "Max =", sol_pip_pass[">400"].max()
print "Median =", sol_pip_pass[">400"].median()
print "Min =", sol_pip_pass[">400"].min()
#lg_bubbles[">400"].value_counts()

Solution up Pipette Shank: concentration in pg/ul of fragments >400 b.p.

Average = 654.8
Standard Deviation = 409.999024389
Standard Error of Mean = 183.357137849
Max = 1368.0
Median = 536.0
Min = 316.0


In [83]:
sp_pass = bubbles[mask6 & bubb_pass]
sp_pass.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
5243,2017-05-17,P2,Gad2-IRES-Cre;Ai14-318381.04.02.01,Nucleated,"Large Bubbles,Solution in Pipette Shank",988.1,494.0,0.5,pass,NaN
5284,2017-05-18,P6,Pvalb-IRES-Cre;Ai14-316465.03.01.04,Partial-Nucleus,"No Bubbles,Solution in Pipette Shank",636.1,316.0,0.5,pass,NaN
5591,2017-06-13,P1,Gad2-IRES-Cre;Ai14-326032.04.01.01,Nucleated,"Fluorescence in Pipette,Cell Dimmed Large Bu...",1297.4,560.0,0.43,pass,NaN
5644,2017-06-19,P2,Slc32a1-IRES-Cre;Ai14-326813.03.02.01,No-Seal,"Fluorescence in Pipette Large Bubbles,Solution...",NaN,NaN,NaN,pass,NaN
5647,2017-06-19,P8,Slc32a1-IRES-Cre;Ai14-326813.04.02.02,Partial-Nucleus,"Fluorescence in Pipette,Cell Dimmed Small Bubb...",2043.0,1368.0,0.67,pass,NaN


# Solution up Pipette Shank pass percentage 

In [84]:
sp_p = sp_pass["Notes"].count()
sp_p
sp_pp = float(sp_p)/sp
sp_pp

0.46153846153846156